In [1]:
import pandas as pd
import yaml
import re
import sys
import os
import math
import datetime

<h1>Library</h1>

In [2]:
DEBUG = False

In [46]:
class SchemaUtils:
    
    def _remove_blanks(column):
        '''
        Utility that returns a filter (i.e., a boolean-valued function that acts on DataFrame rows).
        The filter will 'eliminate' (i.e., return false) for any row for which row[column] is a 'blank',
        i.e., NaN or a string that is empty or just whitespace.
        '''
        def _filter(row):
            val = row[column]
            if type(val) == str and len(val.strip())==0:
                return False
            if type(val) == float and math.isnan(val):
                return False

            return True
        return _filter
    
    def drop_blanks(df, column):
        return df[df.apply(SchemaUtils._remove_blanks(column), axis=1)]

    def to_yaml_date(original_date, BAD_SCHEMA_MSG):
        '''
        Converts the input into another form of date that is suitable to be placed in a YAML document.
        
        For the date, need to convert to a YAML date string 'YYYY-MM-DD', which in turn requires checking
        that value can be formatted into a date (with method 'strftime') to subsequently parse it
        as a datetime object, which from trial and error I found was the date representation that the
        Python yaml module would correctly interpret as a date when creating the YAML document.
        '''
        M                  = SchemaUtils.ValidationMonad(BAD_SCHEMA_MSG)
        converted_date     = M.validate(original_date).has_ducktype_method('Date', 'strftime')    
        converted_date     = converted_date.strftime('%Y-%m-%d') # As required by YAML: ISO-8601 simple date format
        converted_date     = datetime.datetime.strptime(converted_date, '%Y-%m-%d') # Now parse into a datetime
        return converted_date
    
    class ValidationMonad:
        '''
        Monad used to hold context for potentially multiple validations against multiple values.
        When asked to validate a specific value, it returns a more specific Monad that wraps the
        value in question along with the more general context in this Monad.
        '''
        def __init__(self, callerMsg):
            self.callerMsg   = callerMsg

        def validate(self, val):
            return SchemaUtils._ValToValidateMonad(val, self.callerMsg)
        
    class _ValToValidateMonad:
        '''
        This is a 'passthrough' Monad: calling its methods just returns the value wrapped in the Monad,
        unless the value violates the check made by the method in question, in which case it
        raises an exception. The message is a combination of the more contextual message from
        the caller (wrapped in the Monad) and the specific validation violation as per the validation
        method called
        '''
        def __init__(self, val, callerMsg):
            self.val         = val
            self.callerMsg   = callerMsg

        def is_of_type(self, expected_type):
            if type(self.val) != expected_type:
                msg = "For value '" + str(self.val) + "', expected a '" + expected_type.__name__  + "' but instead received a '" + type(self.val).__name__ + "'."
                raise ValueError(self.callerMsg + '\n' + msg)
                
            return self.val
        
        def has_ducktype_method(self, friendly_duck_type_name, method_name):
            if method_name not in dir(self.val):
                val_txt = str(self.val)
                if val_txt == 'nan': # This probably came from Pandas, not from the user who probably 
                                     # left field blank. So for error message, revert to user friendly text
                    val_txt = ''
                msg = "Value '" + val_txt + "', is not a valid " + friendly_duck_type_name  + "."
                raise ValueError(self.callerMsg + '\n' + msg)
                
            return self.val        

In [4]:
class ExcelTableReader:
    '''
    Reads a table of data from an Excel spreadsheet and creates a Pandas DataFrame from it.
    The table is required to have a single row of column headers at the top of the table.
    @param url A path to an Excel spreadsheet with the name of the sheet appended after a colon.
               Example: 'C:/myDocuments/mySpreadsheets/wonderful.xlsx:Sheet1'
    @param excel_range A string representing a range in Excel. The first row must be the column headers.
                       Example: "A3:D10"
    @param horizontally A boolean to determine whether the data to be read is arranged in rows (horizontally=True)
                        or columns (horizontally=False). It is True by default
    @return A Pandas DataFrame built from the data provided.
    '''
    def __init__(self, url, excel_range, horizontally=True):
        self.url          = url
        self.excel_range  = excel_range.upper()
        self.horizontally = horizontally
        
    def read(self, horizontally=True):
        path, sheet                                    = self._parse_url()
        
        first_column, last_column, first_row, last_row = self._parse_range()
        
        if first_row < 1:
            raise ValueError ("Incorrectly formatted Excel range was given: '" + self.excel_range 
                              + "'. Row numbers must be 1 or higher")
            
        if first_row >= last_row:
            raise ValueError ("Incorrectly formatted Excel range was given: '" + self.excel_range 
                              + "'. It spans 0 rows")
        
        # Not that Excel columns start at 1, but Pandas counts rows from 0, so need to offset the
        # header by 1
        if self.horizontally==True:
            header = first_row - 1
            nrows  = last_row - first_row
        else:
            if first_row > 1:
                header = first_row -2
            else:
                header = None 
            nrows  = last_row - first_row +1 # First row is data, not header, so nrows is 1 bigger
        df                                             = pd.read_excel(io         = path, 
                                                                       sheet_name = sheet,
                                                                       header     = header, 
                                                                       usecols    = first_column + ':' + last_column, 
                                                                       nrows      = nrows)
        if DEBUG:
            print("DEBUG --- ExcelTableReader: nrows = " + str(nrows) + ", len(df.index) = " + str(len(df.index)))
        
        if len(df.columns)==0:
            raise ValueError ("Incorrectly formatted Excel range was given: '" + self.excel_range 
                              + "'. It spans no columns with data")
        if len(df.index)==0:
            raise ValueError ("Incorrectly formatted Excel range was given: '" + self.excel_range 
                              + "'. It spans no rows with data")
            
        if self.horizontally==False:
            df = self._rotate(df)
        
        return df
    
    def _parse_url(self):
        '''
        Given a url of form "<some string A, maybe with colons>:<some string B without colons>"
        it returns the two substrings separated by the last colon
        '''
        s = re.split(':', self.url)
        if len(s) < 2:
            raise ValueError ("Incorrectly formatted url was given: '" + self.url
                             +"'. Should instead be formmated like this example: "
                             + "'C:/MyDocuments/MySpreadsheets/Wonderful.xlsx:SheetName'")
        sheet = s[len(s)-1]
        path = self.url.split(':' + sheet)[0]
        if len(path) == 0 or len(sheet) ==0:
            raise ValueError ("Incorrectly formatted url was given: \n\t'" + self.url
                             + "'\nShould instead be formmated like this example, with a non-empty path and a non-empty"
                             + " sheet name separated by the last colon in the url: \n"
                             + "\t'C:/My Documents/My Spreadsheets/Wonderful.xlsx:SheetName'")
        return path, sheet
    
    def _parse_range(self):
        '''
        Parses strings for Excel ranges like 'C5:DA15' and returns the columns and rows: 'C', 'DA', 5, 15.
        If the given range is not correctly formatted then throws an exception
        '''
        REGEX = '^([a-zA-Z]+)([1-9][0-9]*):([a-zA-Z]+)([1-9][0-9]*)$'
        res = re.match(REGEX, self.excel_range)
        if (res == None or len(res.groups()) != 4):
            raise ValueError ("Incorrectly formatted Excel range was given: '" + self.excel_range 
                              + "'. Should instead be formatted like this example: 'C5:DA15'")
            
            
        # Allow for possibility that range is expressed non-monotically. E.g 'B10:D4' instead of 'B4:D10' 
        # So 'correct' user-provided non-monotonic ranges by sorting.
        # Also, to avoid Excel errors, capitalize columns
        # CATCHA: must capitalize before sorting, not the other way around, since a > C so would
        # get wrong result (no columns read). Capitalizing first gives A < C and then you get columns.
        first_column = min(res.group(1).upper(), res.group(3).upper())
        last_column  = max(res.group(1).upper(), res.group(3).upper())
        first_row    = min(int(res.group(2)), int(res.group(4)))
        last_row     = max(int(res.group(2)), int(res.group(4)))
                           
        return first_column, last_column, first_row, last_row
        #return res.group(1), res.group(3), int(res.group(2)), int(res.group(4))
    
    def _rotate(self, df):
        '''
        Used when self.horizontally is false, and we read from Excel that needs to be transposed.
        In that case, the first column of the parameter 'df' is considered to be the columns of the
        rotated dataframe to be computed
        '''
        #df2 = df.dropna()
        keys_p = list(df.columns)[0] # "pointer" to the list of keys, i.e., Excel header for column of keys
        #df2 = df[df.apply(remove_blanks(header_for_list_of_keys), 
        #                  axis=1)] # Drops keys that are NaN, blanks, etc.
        df2 = SchemaUtils.drop_blanks(df, keys_p)
        df2 = df2.set_index(keys_p)
        df2.index.name = None
        df2 = df2.transpose()
        df2 = df2.reset_index()
        df2 = df2.drop(columns=['index'])
        
        return df2       

In [5]:
PRODUCT              = 'product'
JOURNEY              = 'journey'
PLAN_TYPE            = 'planType'
SCENARIO             = 'scenario'
ENVIRONMENT          = 'environment'
SCORING_CYCLE        = 'scoringCycle'
SCORING_MATURITY     = 'scoringMaturity'
ESTIMATED_BY         = 'estimatedBy'
ESTIMATED_ON         = 'estimatedOn'
RECORDED_BY          = 'recordedBy'

CONTEXT_FIELDS = [PRODUCT, JOURNEY, PLAN_TYPE, SCENARIO, ENVIRONMENT,
                  SCORING_CYCLE, SCORING_MATURITY, 
                  ESTIMATED_BY, ESTIMATED_ON, RECORDED_BY]
len(CONTEXT_FIELDS)

10

In [44]:
def readContext(url, excel_range):
    excel_range    = excel_range.upper()
    reader         = ExcelTableReader(url, excel_range, horizontally=False)
    context_df     = reader.read()
    
    # Check context has the right number of rows (which are columns in Excel, since we transposed)
    if len(context_df.index) != 1:
        raise ValueError("Bad Excel range provided: " + excel_range
                        + "\nShould contain exactly two columns: keys and values")
    
    missing_cols = set(CONTEXT_FIELDS).difference(set(context_df.columns))
    if len(missing_cols) > 0:
        missing_txt = ", ".join(["'" + col + "'" for col in missing_cols])
        raise ValueError("Range '" + excel_range + "' lacks these mandatory context fields: "
                        + missing_txt)
        
    BAD_SCHEMA_MSG      = "Incorrect schema for field '" + ESTIMATED_ON + "' when processing the context in range '" \
                            + excel_range + "'."
    M                   = SchemaUtils.ValidationMonad(BAD_SCHEMA_MSG)
        
    ctx = {}
    for field in CONTEXT_FIELDS:
        ctx[field] = context_df.iloc[0][field]
        
    # Validations for some fields
    ctx[ESTIMATED_ON]  = SchemaUtils.to_yaml_date(ctx[ESTIMATED_ON], BAD_SCHEMA_MSG)
    
    return ctx   

In [25]:
def applyMarathonJourneyPlan(ctx, url, excel_range, repo_root_dir):
    product             = ctx[PRODUCT]
    scoring_cycle       = ctx[SCORING_CYCLE]
    plan_maturity       = ctx[SCORING_MATURITY]
    environment         = ctx[ENVIRONMENT]
    planning_scenario   = ctx[SCENARIO]
    estimating_mgr      = ctx[ESTIMATED_BY]
    user                = ctx[RECORDED_BY]
    plan_type           = ctx[PLAN_TYPE]
    journey             = ctx[JOURNEY]
    
    BAD_SCHEMA_MSG      = "Bad estimation date provided in context"
    estimation_date     = SchemaUtils.to_yaml_date(ctx[ESTIMATED_ON], BAD_SCHEMA_MSG)
    
    reader              = ExcelTableReader(url, excel_range)
    
    plan_df             = reader.read()
    if len(plan_df.columns) != 2:
        raise ValueError ("Badly formatted Marathon Plan: should have exactly two columns, ideally called: 'Workstream' and "
                         + "'Effort'. Error when processing range= '" + excel_range + "' and url=\n\t" + url)
    plan_df.columns     = ['Workstream', 'Effort']
    
    # Drop workstreams that were not defined
    plan_df             = SchemaUtils.drop_blanks(plan_df, 'Workstream')
   
    
    manifest_dict       = {}
    workstreams         = []
    WORKSTREAM_ID       = 1
    
    BAD_SCHEMA_MSG      = "Incorrect schema for a Marathon Plan in range '" + excel_range + "'."
    M                   = SchemaUtils.ValidationMonad(BAD_SCHEMA_MSG)
    
    for row in plan_df.iterrows():
        workstream      = row[1]['Workstream']
        effort          = row[1]['Effort']
        effort          = M.validate(effort).is_of_type(float)
        
        workstreams.append({'workstream'   : workstream, 
                            'effort'       : effort, 
                            'workstream-id': 'ws-' + str(WORKSTREAM_ID)})
        WORKSTREAM_ID += 1

    # Namespae would typically be something like 'Development' or 'Production'
    metadata      = {'namespace': environment + '.' + scoring_cycle, 
                     'name': product + '.' + journey + '.' + planning_scenario,
                     'labels': {'product': product, 'scoringCycle': scoring_cycle, 'scenario': planning_scenario,
                                                  'journey': journey}}

    manifest_dict['apiVersion']     = 'journeys.inbound.a6i.io/v1dev'
    manifest_dict['kind']           = 'JourneyPlan'
    manifest_dict['metadata']       = metadata
    # Plan maturity can be one of: 'Not done', 'Drafted', 'Checked', 'Published'
    manifest_dict['planMaturity']   = plan_maturity
    manifest_dict['plan']           = {'type': plan_type, 
                                       'estimated_by': estimating_mgr, 
                                        'estimated_on': estimation_date,
                                        'recorded_by': user,
                                       'workstreams': workstreams}    
    
    yaml.dump(manifest_dict, sys.stdout)
    
    with open(repo_root_dir + '/' + product + '-marathon-plan.yaml', 'w') as file:
        yaml.dump(manifest_dict, file)

In [66]:
def applyInvestmentCommittment(ctx, url, excel_range, repo_root_dir):
    
    product             = ctx[PRODUCT]
    scoring_cycle       = ctx[SCORING_CYCLE]
    plan_maturity       = ctx[SCORING_MATURITY]
    environment         = ctx[ENVIRONMENT]
    planning_scenario   = ctx[SCENARIO]
    committing_mgr      = ctx[ESTIMATED_BY]
    user                = ctx[RECORDED_BY]
    plan_type           = ctx[PLAN_TYPE]
    journey             = ctx[JOURNEY]
    
    BAD_SCHEMA_MSG      = "Bad estimation date provided in context"
    committing_date     = SchemaUtils.to_yaml_date(ctx[ESTIMATED_ON], BAD_SCHEMA_MSG)
    
    # Load data and validate its geometric shape
    reader        = ExcelTableReader(url, excel_range)
    
    plan_df       = reader.read()
    if len(plan_df.columns) != 2:
        raise ValueError ("Badly formatted Investment Plan: should have exactly two columns, "\
                          + "ideally called: 'Period' and Investment'. "\
                          + "Error when processing range= '" + excel_range + "' and url=\n\t" + url)
    plan_df.columns = ['Period', 'Investment']
    
    # Drop workstreams that were not defined
    plan_df             = SchemaUtils.drop_blanks(plan_df, 'Period')
    
    manifest_dict = {}
    investment_ts   = []
    BAD_SCHEMA_MSG      = "Incorrect schema for a Investment Plan in range '" + excel_range + "'."
    M                   = SchemaUtils.ValidationMonad(BAD_SCHEMA_MSG)
    
    for row in plan_df.iterrows():
        period          = row[1]['Period']
        investment      = row[1]['Investment']
        investment      = M.validate(investment).is_of_type(float)
        investment_ts.append({'period'      : period, 
                              'investment'  : investment,
                              'units'       : 'person-days'})

    # Namespace would typically be something like 'Development' or 'Production'
    metadata      = {'namespace': environment + '.' + scoring_cycle, 
                     'name'     : product + '.' + journey + '.' + planning_scenario,
                     'labels'   : {'product': product, 
                                   'scoringCycle': scoring_cycle, 
                                   'scenario': planning_scenario,
                                   'journey': journey}}

    manifest_dict['apiVersion']     = 'journeys.inbound.a6i.io/v1dev'
    manifest_dict['kind']           = 'JourneyInvestment'
    manifest_dict['metadata']       = metadata
    # Plan maturity can be one of: 'Not done', 'Drafted', 'Checked', 'Published'
    manifest_dict['planMaturity']   = plan_maturity
    manifest_dict['committment']           = {'committed_by': committing_mgr, 
                                              'committed_on': committing_date,
                                              'recorded_by': user,
                                              'investment': investment_ts}    
    
    yaml.dump(manifest_dict, sys.stdout)
    
    with open(repo_root_dir + '/' + product + '-investment-committment.yaml', 'w') as file:
        yaml.dump(manifest_dict, file)

<h1>Tests</h1>

<h3>API: marathon-investment.xlapi.a6s.io</h3>

In [55]:
XL_POSTINGS_DIR = os.getcwd() + '/../excel-api'
POSTING_FILE    = 'marathon-investment.xlapi.a6s.xlsx'
URL = XL_POSTINGS_DIR + '/' + POSTING_FILE + ':' + 'journey-investment'

In [56]:
#DEBUG = True
r1 = ExcelTableReader(URL, 'E2:F9')
df1 = r1.read()
df1

,Big Rock,T-Shirt Size Effort (person-days)
0,End-to-end support for UXP,3000.0
1,Migrate from Windows to Linux,2000.0
2,Containerization in Linux,1500.0
3,Cloud stabilization,300.0
4,Confirmation migration away for Access,1000.0
5,,NaN
6,CI/CD adoption via Anthos,1500.0


In [57]:
r2 = ExcelTableReader(URL, 'B2:c10', horizontally=False)
df2 = r2.read()
df2

,product,journey,planType,scenario,environment,scoringCycle,scoringMaturity,estimatedBy,estimatedOn
0,MyTransformingProduct,Modernization,Marathon,Default,Production,Dec 2020,Draft,joe.thedevhead@mycorp.com,2015-12-01


In [58]:
CTX = readContext(URL, 'b11:c1')
CTX

{'product': 'MyTransformingProduct',
 'journey': 'Modernization',
 'planType': 'Marathon',
 'scenario': 'Default',
 'environment': 'Production',
 'scoringCycle': 'Dec 2020',
 'scoringMaturity': 'Draft',
 'estimatedBy': 'joe.thedevhead@mycorp.com',
 'estimatedOn': datetime.datetime(2015, 12, 1, 0, 0),
 'recordedBy': 'jill.thearchitect@mycorp.com'}

In [59]:
MANIFESTS_REPO = os.getcwd() + '/../test/manifests-repo'
MANIFESTS_REPO

'C:\\Users\\aleja\\Documents\\Code\\chateauclaudia-labs\\apodeixi\\project\\controllers/../test/manifests-repo'

In [60]:
PLAN_RANGE = 'E2:F10'
applyMarathonJourneyPlan (ctx=CTX, url=URL, excel_range=PLAN_RANGE, repo_root_dir=MANIFESTS_REPO)

apiVersion: journeys.inbound.a6i.io/v1dev
kind: JourneyPlan
metadata:
  labels:
    journey: Modernization
    product: MyTransformingProduct
    scenario: Default
    scoringCycle: Dec 2020
  name: MyTransformingProduct.Modernization.Default
  namespace: Production.Dec 2020
plan:
  estimated_by: joe.thedevhead@mycorp.com
  estimated_on: 2015-12-01 00:00:00
  recorded_by: jill.thearchitect@mycorp.com
  type: Marathon
  workstreams:
  - effort: 3000.0
    workstream: End-to-end support for UXP
    workstream-id: ws-1
  - effort: 2000.0
    workstream: Migrate from Windows to Linux
    workstream-id: ws-2
  - effort: 1500.0
    workstream: Containerization in Linux
    workstream-id: ws-3
  - effort: 300.0
    workstream: Cloud stabilization
    workstream-id: ws-4
  - effort: 1000.0
    workstream: Confirmation migration away for Access
    workstream-id: ws-5
  - effort: 1500.0
    workstream: CI/CD adoption via Anthos
    workstream-id: ws-6
  - effort: 3000.0
    workstream: Decompos

In [67]:
INVESTMENT_RANGE = 'H3:I9'
applyInvestmentCommittment (ctx=CTX, url=URL, excel_range=INVESTMENT_RANGE, repo_root_dir=MANIFESTS_REPO)

apiVersion: journeys.inbound.a6i.io/v1dev
committment:
  committed_by: joe.thedevhead@mycorp.com
  committed_on: 2015-12-01 00:00:00
  investment:
  - investment: 2400.0
    period: FY 2021
    units: person-days
  - investment: 3000.0
    period: FY 2022
    units: person-days
  - investment: 2400.0
    period: FY 2023
    units: person-days
  - investment: 2400.0
    period: FY 2024
    units: person-days
  - investment: 2100.0
    period: FY 2025
    units: person-days
  recorded_by: jill.thearchitect@mycorp.com
kind: JourneyInvestment
metadata:
  labels:
    journey: Modernization
    product: MyTransformingProduct
    scenario: Default
    scoringCycle: Dec 2020
  name: MyTransformingProduct.Modernization.Default
  namespace: Production.Dec 2020
planMaturity: Draft


<h1> SCRAP </h1>